<h1> DenseNet-121 Training — Interpretability-Focused Pipeline</h1>

<p>
This training block fine-tunes a <strong>DenseNet-121 model</strong> on the cleaned chest X-ray dataset while preserving visual interpretability of attention maps.
Unlike performance-only pipelines, this setup avoids heavy or unrealistic augmentations and <strong>trains the model in a way that produces trustworthy Grad-CAM / Grad-CAM++ maps</strong>.
</p>

<h3>✔ Key Training Decisions</h3>
<ul>
  <li>Grayscale to 3-channel replication (stable for medical models)</li>
  <li>Only <strong>safe augmentations</strong> (no mixup, cutout, blur, noise)</li>
  <li><strong>Full model fine-tuning</strong> for best saliency/attribution quality</li>
  <li>Binary objective: Pneumonia vs Normal (<code>BCEWithLogitsLoss</code>)</li>
  <li><strong>Best checkpoint saved using validation AUC</strong></li>
</ul>

<h3>Validation Metrics Used</h3>
<ul>
  <li>AUROC (primary)</li>
  <li>F1-Score</li>
</ul>

<h3>Final Workflow Summary</h3>
<ol>
  <li>Load cleaned dataset (no “R” marker)</li>
  <li>Split into Train / Validation / Test</li>
  <li>Train DenseNet-121 end-to-end with Adam</li>
  <li>Save only the best model based on <strong>highest validation AUROC</strong></li>
  <li>Evaluate on the test set using the saved checkpoint</li>
</ol>

#Setup (Imports + Device + Drive + Unzip)

In [ ]:
import os, shutil, numpy as np, torch, torch.nn as nn, torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, f1_score
from PIL import Image
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, classification_report, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

from google.colab import drive
drive.mount('/content/drive')
!unzip "/content/drive/MyDrive/X_ray_images/trust.zip" -d "/content/"

orig_train = "/content/chest_xray/train"
orig_test  = "/content/chest_xray/test"
clean_root = "/content/chest_xray_cleaned"
clean_train = f"{clean_root}/train"
clean_test  = f"{clean_root}/test"

os.makedirs(clean_train, exist_ok=True)
os.makedirs(clean_test, exist_ok=True)


#Remove “R” Marker + Clean Dataset

In [ ]:
def remove_R(np_img):
    if len(np_img.shape) == 3:
        np_img = cv2.cvtColor(np_img, cv2.COLOR_BGR2GRAY)
    gray = np_img.copy()
    _, thresh = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.zeros_like(gray)
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        area = cv2.contourArea(c)
        if 15 < w < 120 and 15 < h < 120 and 50 < area < 6000:
            cv2.drawContours(mask, [c], -1, 255, -1)
    cleaned = cv2.inpaint(gray, mask, 5, cv2.INPAINT_TELEA)
    return cleaned

def preprocess_folder(src, dst):
    for class_name in ["NORMAL", "PNEUMONIA"]:
        src_dir = os.path.join(src, class_name)
        dst_dir = os.path.join(dst, class_name)
        os.makedirs(dst_dir, exist_ok=True)
        for f in tqdm(os.listdir(src_dir), desc=f"Cleaning {class_name}"):
            if f.lower().endswith((".jpg", ".jpeg", ".png")):
                in_path  = os.path.join(src_dir, f)
                out_path = os.path.join(dst_dir, f)
                img = Image.open(in_path).convert("L")
                cleaned = remove_R(np.array(img))
                Image.fromarray(cleaned).save(out_path)

print("\n=== CLEANING TRAIN SET ===")
preprocess_folder(orig_train, clean_train)
print("\n=== CLEANING TEST SET ===")
preprocess_folder(orig_test, clean_test)
print("\nCleaning complete.")


#Train/Val Split from Cleaned Data

In [ ]:
new_train = "/content/chest_xray_cleaned/new_train"
new_val   = "/content/chest_xray_cleaned/new_val"

os.makedirs(new_train, exist_ok=True)
os.makedirs(new_val, exist_ok=True)
os.makedirs(f"{new_train}/NORMAL", exist_ok=True)
os.makedirs(f"{new_train}/PNEUMONIA", exist_ok=True)
os.makedirs(f"{new_val}/NORMAL", exist_ok=True)
os.makedirs(f"{new_val}/PNEUMONIA", exist_ok=True)

normal_imgs = [os.path.join(clean_train, "NORMAL", f) for f in os.listdir(os.path.join(clean_train, "NORMAL"))]
pneu_imgs   = [os.path.join(clean_train, "PNEUMONIA", f) for f in os.listdir(os.path.join(clean_train, "PNEUMONIA"))]

train_norm, val_norm = train_test_split(normal_imgs, test_size=0.15, random_state=42)
train_pneu, val_pneu = train_test_split(pneu_imgs, test_size=0.15, random_state=42)

def copy_list(files, dest):
    for f in files:
        shutil.copy(f, dest)

copy_list(train_norm, f"{new_train}/NORMAL")
copy_list(val_norm,   f"{new_val}/NORMAL")
copy_list(train_pneu, f"{new_train}/PNEUMONIA")
copy_list(val_pneu,   f"{new_val}/PNEUMONIA")

print("\nTrain/Val split ready.")


#Transforms(Train/Test)

In [ ]:
IMG_SIZE = 224

train_tfms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

test_tfms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])


#Datasets+Loaders

In [ ]:
train_ds = datasets.ImageFolder(new_train, transform=train_tfms)
val_ds   = datasets.ImageFolder(new_val,   transform=test_tfms)
test_ds  = datasets.ImageFolder(clean_test, transform=test_tfms)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=16, shuffle=False)


#Model + Optimizer

In [ ]:
model = models.densenet121(weights="IMAGENET1K_V1")
model.classifier = nn.Linear(model.classifier.in_features, 1)
model = model.to(device)

for p in model.parameters():
    p.requires_grad = True

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)


#Train + Validation Functions

In [ ]:
def train_epoch():
    model.train()
    total_loss = 0
    for imgs, labels in tqdm(train_loader):
        imgs = imgs.to(device)
        labels = labels.float().unsqueeze(1).to(device)
        optimizer.zero_grad()
        out = model(imgs)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def evaluate(loader):
    model.eval()
    probs, trues = [], []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            out = model(imgs)
            p = torch.sigmoid(out).cpu().numpy().flatten()
            probs.extend(p)
            trues.extend(labels.numpy())
    probs = np.array(probs)
    trues = np.array(trues)
    auc = roc_auc_score(trues, probs)
    preds = (probs > 0.5).astype(int)
    f1 = f1_score(trues, preds)
    return auc, f1


#Training Loop (Save Best)

In [ ]:
best_auc = 0
save_dir = "/content/saved_models"
os.makedirs(save_dir, exist_ok=True)
best_path = f"{save_dir}/best_interpretability_model.pth"

EPOCHS = 15

print("\nStarting Training (Interpretability Mode ON)...\n")

for epoch in range(1, EPOCHS + 1):
    print(f"\nEpoch {epoch}/{EPOCHS}")
    train_loss = train_epoch()
    val_auc, val_f1 = evaluate(val_loader)

    print(f"Train Loss : {train_loss:.4f}")
    print(f"Val AUC    : {val_auc:.4f}")
    print(f"Val F1     : {val_f1:.4f}")

    if val_auc > best_auc:
        best_auc = val_auc
        torch.save(model.state_dict(), best_path)
        print(f" New BEST model saved → {best_path}")

print("\nTraining Complete!")
print("Best AUC:", best_auc)
print("Saved Best Model:", best_path)


#Test Evaluation Summary

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_curve

print(f"Loading weights from: {best_path}")
checkpoint = torch.load(best_path, map_location=device)
model.load_state_dict(checkpoint)
model.eval()

y_true = []
y_probs = []

print("Running inference on Test Set...")
with torch.no_grad():
    for imgs, labels in tqdm(test_loader, desc="Testing"):
        imgs = imgs.to(device)
        out = model(imgs)
        probs = torch.sigmoid(out).cpu().numpy().flatten()
        y_probs.extend(probs)
        y_true.extend(labels.numpy())

y_true = np.array(y_true)
y_probs = np.array(y_probs)
y_pred = (y_probs > 0.5).astype(int)

auc = roc_auc_score(y_true, y_probs)
f1 = f1_score(y_true, y_pred)
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print("\n" + "="*30)
print(" 🩺 FINAL TEST RESULTS")
print("="*30)
print(f"AUROC       : {auc:.4f}")
print(f"F1 Score    : {f1:.4f}")
print(f"Accuracy    : {(tp+tn)/len(y_true):.4f}")
print(f"Sensitivity : {sensitivity:.4f}")
print(f"Specificity : {specificity:.4f}")
print("="*30)
print("\nDetailed Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=["NORMAL", "PNEUMONIA"]))


#Confusion Matrix & ROC Plot

In [ ]:
plt.figure(figsize=(6, 5))
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=["Normal", "Pneumonia"],
            yticklabels=["Normal", "Pneumonia"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

fpr, tpr, thresholds = roc_curve(y_true, y_probs)
plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()


#Compute Optimal Threshold and reevaluate with new Threshold

In [ ]:
fpr, tpr, thresholds = roc_curve(y_true, y_probs)
J = tpr - fpr
ix = np.argmax(J)
best_thresh = thresholds[ix]

print(f"Checking {len(thresholds)} possible thresholds...")
print(f"🚀 Best Threshold Found: {best_thresh:.4f}")


In [ ]:
y_pred_new = (y_probs > best_thresh).astype(int)
tn, fp, fn, tp = confusion_matrix(y_true, y_pred_new).ravel()
new_acc = (tp + tn) / len(y_true)
new_sens = tp / (tp + fn)
new_spec = tn / (tn + fp)


In [ ]:
print("\n" + "="*30)
print(" ⚖️  BALANCED RESULTS")
print("="*30)
print(f"Old Accuracy : {0.8125:.4f} -> New Accuracy : {new_acc:.4f}")
print(f"Sensitivity  : {0.9974:.4f} -> New Sens     : {new_sens:.4f}")
print(f"Specificity  : {0.5043:.4f} -> New Spec     : {new_spec:.4f}")
print("="*30)
